In [1]:
%pip install beacon-api==1.0.9rc2


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from beacon_api import *
client = Client("https://beacon-emod-chem.maris.nl", jwt_token="<your_jwt_token_here>")

Connected to: https://beacon-emod-chem.maris.nl/ server successfully


In [3]:
tables = client.list_tables()
tables

{'emodnet_chemistry': <beacon_api.table.DataTable at 0x7f8e58361f10>,
 'default': <beacon_api.table.DataTable at 0x7f8de151ef30>}

In [4]:
tables['default'].get_table_schema()

Cruise: string
Station: string
Type: string
yyyy-mm-ddThh:mm:ss.sss: timestamp[ms]
Longitude: float
Latitude: float
LOCAL_CDI_ID: string
EDMO_code: int64
Bot. Depth: float
Instrument Info: string
Codes in Originator File: string
P35 Contributor Codes: string
References: string
Comments: string
Data set name: string
Discipline: string
Category: string
Variables measured: string
Data format: string
Data format version: string
Data size: string
Data set creation date: string
Datum: string
Measuring area type: string
Water depth: float
Depth reference: string
Minimum instrument depth: float
Maximum instrument depth: float
Start date: string
Start time: string
End date: string
End time: string
Vertical resolution: string
Vertical resolution unit: string
Instrument / gear type: string
Track resolution: string
Track resolution unit: string
Frequency: string
Frequency unit: string
Platform type: string
Cruise name: string
Alternative cruise name: string
Cruise start date: string
Station name: 

In [5]:
query_builder = tables['default'].query()
query_builder = (
    query_builder
    .add_select_coalesced(["time_ISO8601", "yyyy-mm-ddThh:mm:ss.sss"], "COMMON_TIME")
    .add_select_column("time_ISO8601_qc", "COMMON_TIME_QC")
    .add_select_column("yyyy-mm-ddThh:mm:ss.sss")
    .add_select_column("Longitude")
    .add_select_column("Longitude.units")
    .add_select_column("Longitude", "COMMON_LONGITUDE")
    .add_select(SelectLiteral("degree_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    .add_select_column("Latitude")
    .add_select_column("Latitude.units")
    .add_select_column("Latitude", "COMMON_LATITUDE")
    .add_select(SelectLiteral("degree_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    .add_select_column("Depth")
    .add_select_column("Depth_qc")
    .add_select_column("Depth.units")
    .add_select_column("Depth", "COMMON_ORIGIN_DEPTH")
    .add_select_column("Depth_qc", "COMMON_ORIGIN_DEPTH_QC")
    .add_select_column("Depth.units", "COMMON_ORIGIN_DEPTH_UNITS")
    .add_select(SelectLiteral("", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_ORIGIN_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_ORIGIN_DEPTH_P06"))
    .add_select_column("Depth", "COMMON_DEPTH")
    .add_select_column("Depth_qc", "COMMON_DEPTH_QC")
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
)


In [6]:
query_builder = (
    query_builder
    .add_select(SelectLiteral("", "COMMON_EDMERP_CODE"))
    .add_select_column("EDMO_code", "COMMON_EDMO_CODE")
    .add_select_column("CSR platform code", "COMMON_CSR")
    .add_select(SelectFunction("concat", [SelectColumn("EDMO_code"), SelectColumn("LOCAL_CDI_ID")], "COMMON_ODV_TAG"))
    .add_select(SelectLiteral("BEACON_EMODNET_CHEMISTRY", "SOURCE_BDI"))
    .add_select(SelectFunction("concat", [SelectColumn("EDMO_code"), SelectColumn("LOCAL_CDI_ID")], "SOURCE_BDI_DATASET_ID"))
    .add_select(SelectFunction("nvl2", [
        SelectColumn("CSR platform code"),
        SelectFunction("concat", [
            SelectLiteral("SDN:C17::"),
            SelectColumn("CSR platform code")
            ]),
        SelectLiteral("")
        ],
        alias="COMMON_PLATFORM_C17"))
    .add_select(SelectFunction("map_emodnet_chemistry_platform_l06", [SelectColumn("Platform type")], "COMMON_PLATFORM_L06"))
    .add_select_column("Cruise")
    .add_select_column("Cruise name")
    .add_select_column("CSR platform code")
    .add_select_column("CSR platform name")
    .add_select_column("Measuring area type")
    .add_select_column("Platform type")
    .add_select_column("Instrument / gear type")
    .add_select_column("LOCAL_CDI_ID")
    .add_select_column("EDMO_code")
    .add_select_column("P35 Contributor Codes")
    .add_select_column("featureType")
    .add_select_column("featureType", "COMMON_FEATURE_TYPE")
)

In [7]:
query_builder = (
    query_builder
    .add_select_column("Water body chlorophyll-a")
    .add_select_column("Water body chlorophyll-a_qc")
    .add_select_column("Water body chlorophyll-a.units")
    .add_select(SelectColumn("Water body chlorophyll-a", "COMMON_ORIGIN_CHLOROPHYLL"))
    .add_select(SelectColumn("Water body chlorophyll-a_qc", "COMMON_ORIGIN_CHLOROPHYLL_QC"))
    .add_select(SelectLiteral("Water body chlorophyll-a", "COMMON_ORIGIN_CHLOROPHYLL_STANDARD_NAME"))
    .add_select(SelectColumn("Water body chlorophyll-a.units", "COMMON_ORIGIN_CHLOROPHYLL_UNITS"))
    .add_select(SelectLiteral("SDN:P01::CHLTVOLU", "COMMON_ORIGIN_CHLOROPHYLL_P01"))
    .add_select(SelectLiteral("SDN:P06::UMMC", "COMMON_ORIGIN_CHLOROPHYLL_P06"))
    .add_select(SelectColumn("Water body chlorophyll-a", "COMMON_CHLOROPHYLL_PER_VOLUME"))
    .add_select(SelectColumn("Water body chlorophyll-a_qc", "COMMON_CHLOROPHYLL_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mass_concentration_of_chlorophyll_in_sea_water", "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("mg m-3", "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::CHLTVOLU", "COMMON_CHLOROPHYLL_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UMMC", "COMMON_CHLOROPHYLL_PER_VOLUME_P06"))
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_CHLOROPHYLL_L05"))
    .add_select(SelectLiteral("", "COMMON_CHLOROPHYLL_L22"))
    .add_select(SelectLiteral("", "COMMON_CHLOROPHYLL_L33"))
)

In [8]:
# Oxygen
query_builder = (
    query_builder
    .add_select_column("Water body dissolved oxygen concentration")
    .add_select_column("Water body dissolved oxygen concentration_qc")
    .add_select_column("Water body dissolved oxygen concentration.units")
    .add_select(SelectColumn("Water body dissolved oxygen concentration", "COMMON_ORIGIN_OXYGEN"))
    .add_select(SelectColumn("Water body dissolved oxygen concentration_qc", "COMMON_ORIGIN_OXYGEN_QC"))
    .add_select(SelectLiteral("Water body dissolved oxygen concentration", "COMMON_ORIGIN_OXYGEN_STANDARD_NAME"))
    .add_select(SelectColumn("Water body dissolved oxygen concentration.units", "COMMON_ORIGIN_OXYGEN_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXYZZXX", "COMMON_ORIGIN_OXYGEN_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_OXYGEN_P06"))
    # Per volume
    .add_select(SelectColumn("Water body dissolved oxygen concentration", "COMMON_OXYGEN_PER_VOLUME"))
    .add_select(SelectColumn("Water body dissolved oxygen concentration_qc", "COMMON_OXYGEN_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_dissolved_molecular_oxygen_in_sea_water", "COMMON_OXYGEN_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_OXYGEN_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXYZZXX", "COMMON_OXYGEN_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_OXYGEN_PER_VOLUME_P06"))
    
    # Per mass
    .add_select(
        SelectFunction("map_units", [
                SelectLiteral("SDN:P06::UPOX"), 
                SelectLiteral("SDN:P06::KGUM"), 
                SelectColumn("Water body dissolved oxygen concentration")
            ],
            "COMMON_OXYGEN_PER_MASS")
        )
    .add_select(SelectColumn("Water body dissolved oxygen concentration_qc", "COMMON_OXYGEN_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_oxygen_per_unit_mass_in_sea_water", "COMMON_OXYGEN_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_OXYGEN_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXMZZXX", "COMMON_OXYGEN_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_OXYGEN_PER_MASS_P06"))
    # Instruments
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_OXYGEN_L05"))
    .add_select(SelectLiteral("", "COMMON_OXYGEN_L22"))
    .add_select(SelectLiteral("", "COMMON_OXYGEN_L33"))
)

In [9]:
# Nitrate
query_builder = (
    query_builder
    .add_select_column("Water body nitrate")
    .add_select_column("Water body nitrate_qc")
    .add_select_column("Water body nitrate.units")
    .add_select(SelectColumn("Water body nitrate", "COMMON_ORIGIN_NITRATE"))
    .add_select(SelectColumn("Water body nitrate_qc", "COMMON_ORIGIN_NITRATE_QC"))
    .add_select(SelectLiteral("Water body nitrate", "COMMON_ORIGIN_NITRATE_STANDARD_NAME"))
    .add_select(SelectColumn("Water body nitrate.units", "COMMON_ORIGIN_NITRATE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::NTRAZZXX", "COMMON_ORIGIN_NITRATE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_NITRATE_P06"))
    # Per volume
    .add_select(SelectColumn("Water body nitrate", "COMMON_NITRATE_PER_VOLUME"))
    .add_select(SelectColumn("Water body nitrate_qc", "COMMON_NITRATE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_nitrate_in_sea_water", "COMMON_NITRATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_NITRATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::NTRAZZXX", "COMMON_NITRATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_NITRATE_PER_VOLUME_P06"))
    # Per mass
    .add_select(SelectFunction("map_units", [
            SelectLiteral("SDN:P06::UPOX"), 
            SelectLiteral("SDN:P06::KGUM"), 
            SelectColumn("Water body nitrate")
        ],
        "COMMON_NITRATE_PER_MASS")
    )
    .add_select(SelectColumn("Water body nitrate_qc", "COMMON_NITRATE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_nitrate_per_unit_mass_in_sea_water", "COMMON_NITRATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_NITRATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP005", "COMMON_NITRATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_NITRATE_PER_MASS_P06"))
    # Instruments
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_NITRATE_L05"))
    .add_select(SelectLiteral("", "COMMON_NITRATE_L22"))
    .add_select(SelectLiteral("", "COMMON_NITRATE_L33"))
)

In [10]:
# Nitrate Nitrite
query_builder = (
    query_builder
    .add_select_column("Water body nitrate plus nitrite")
    .add_select_column("Water body nitrate plus nitrite_qc")
    .add_select_column("Water body nitrate plus nitrite.units")
    .add_select(SelectColumn("Water body nitrate plus nitrite", "COMMON_ORIGIN_NITRATE_NITRITE"))
    .add_select(SelectColumn("Water body nitrate plus nitrite_qc", "COMMON_ORIGIN_NITRATE_NITRITE_QC"))
    .add_select(SelectColumn("Water body nitrate plus nitrite.units", "COMMON_ORIGIN_NITRATE_NITRITE_UNITS"))
    .add_select(SelectLiteral("Water body nitrate plus nitrite", "COMMON_ORIGIN_NITRATE_NITRITE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::NTRZZZXX", "COMMON_ORIGIN_NITRATE_NITRITE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_NITRATE_NITRITE_P06"))
    # Per volume
    .add_select(SelectColumn("Water body nitrate plus nitrite", "COMMON_NITRATE_NITRITE_PER_VOLUME"))
    .add_select(SelectColumn("Water body nitrate plus nitrite_qc", "COMMON_NITRATE_NITRITE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_nitrate_and_nitrite_in_sea_water", "COMMON_NITRATE_NITRITE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_NITRATE_NITRITE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::NTRZZZXX", "COMMON_NITRATE_NITRITE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_NITRATE_NITRITE_PER_VOLUME_P06"))
    # Per mass
    .add_select(SelectFunction("map_units", [
            SelectLiteral("SDN:P06::UPOX"), 
            SelectLiteral("SDN:P06::KGUM"), 
            SelectColumn("Water body nitrate plus nitrite")
        ],
        "COMMON_NITRATE_NITRITE_PER_MASS")
    )
    .add_select(SelectColumn("Water body nitrate plus nitrite_qc", "COMMON_NITRATE_NITRITE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_nitrate_and_nitrite_per_unit_mass_in_sea_water", "COMMON_NITRATE_NITRITE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_NITRATE_NITRITE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP006", "COMMON_NITRATE_NITRITE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_NITRATE_NITRITE_PER_MASS_P06"))
    # Instruments
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_NITRATE_NITRITE_L05"))
    .add_select(SelectLiteral("", "COMMON_NITRATE_NITRITE_L22"))
    .add_select(SelectLiteral("", "COMMON_NITRATE_NITRITE_L33"))
)

In [11]:
# Map ammonium
query_builder = (
    query_builder
    .add_select_column("Water body ammonium")
    .add_select_column("Water body ammonium_qc")
    .add_select_column("Water body ammonium.units")
    
    .add_select(SelectColumn("Water body ammonium", "COMMON_ORIGIN_AMMONIUM"))
    .add_select(SelectColumn("Water body ammonium_qc", "COMMON_ORIGIN_AMMONIUM_QC"))
    .add_select(SelectLiteral("Water body ammonium", "COMMON_ORIGIN_AMMONIUM_STANDARD_NAME"))
    .add_select(SelectColumn("Water body ammonium.units", "COMMON_ORIGIN_AMMONIUM_UNITS"))
    .add_select(SelectLiteral("SDN:P01::AMONZZXX", "COMMON_ORIGIN_AMMONIUM_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_AMMONIUM_P06"))
    # Per volume
    .add_select(SelectColumn("Water body ammonium", "COMMON_AMMONIUM_PER_VOLUME"))
    .add_select(SelectColumn("Water body ammonium_qc", "COMMON_AMMONIUM_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_ammonium_in_sea_water", "COMMON_AMMONIUM_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_AMMONIUM_PER_VOLUME_UNITS"))
    .add_select (SelectLiteral("SDN:P01::AMONZZXX", "COMMON_AMMONIUM_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_AMMONIUM_PER_VOLUME_P06"))
    # Per mass
    .add_select(
        SelectFunction("map_units", [
                SelectLiteral("SDN:P06::UPOX"), 
                SelectLiteral("SDN:P06::KGUM"), 
                SelectColumn("Water body ammonium")
            ],
            "COMMON_AMMONIUM_PER_MASS")
        )
    .add_select(SelectColumn("Water body ammonium_qc", "COMMON_AMMONIUM_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_ammonium_per_unit_mass_in_sea_water", "COMMON_AMMONIUM_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_AMMONIUM_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP004", "COMMON_AMMONIUM_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_AMMONIUM_PER_MASS_P06"))
    # Instruments
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_AMMONIUM_L05"))
    .add_select(SelectLiteral("", "COMMON_AMMONIUM_L22"))
    .add_select(SelectLiteral("", "COMMON_AMMONIUM_L33"))
)

In [12]:
# Phosphate
query_builder = (
    query_builder
    .add_select_column("Water body phosphate")
    .add_select_column("Water body phosphate_qc")
    .add_select_column("Water body phosphate.units")
    .add_select(SelectColumn("Water body phosphate", "COMMON_ORIGIN_PHOSPHATE"))
    .add_select(SelectColumn("Water body phosphate_qc", "COMMON_ORIGIN_PHOSPHATE_QC"))
    .add_select(SelectLiteral("Water body phosphate", "COMMON_ORIGIN_PHOSPHATE_STANDARD_NAME"))
    .add_select(SelectColumn("Water body phosphate.units", "COMMON_ORIGIN_PHOSPHATE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PHOSZZXX", "COMMON_ORIGIN_PHOSPHATE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_PHOSPHATE_P06"))
    # Per volume
    .add_select(SelectColumn("Water body phosphate", "COMMON_PHOSPHATE_PER_VOLUME"))
    .add_select(SelectColumn("Water body phosphate_qc", "COMMON_PHOSPHATE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_phosphate_in_sea_water", "COMMON_PHOSPHATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_PHOSPHATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PHOSZZXX", "COMMON_PHOSPHATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_PHOSPHATE_PER_VOLUME_P06"))
    # Per mass
    .add_select(
        SelectFunction("map_units", [
                SelectLiteral("SDN:P06::UPOX"), 
                SelectLiteral("SDN:P06::KGUM"), 
                SelectColumn("Water body phosphate")
            ],
            "COMMON_PHOSPHATE_PER_MASS")
    )
    .add_select(SelectColumn("Water body phosphate_qc", "COMMON_PHOSPHATE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_phosphate_per_unit_mass_in_sea_water", "COMMON_PHOSPHATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_PHOSPHATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP906", "COMMON_PHOSPHATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_PHOSPHATE_PER_MASS_P06"))
    # Instruments
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_PHOSPHATE_L05"))
    .add_select(SelectLiteral("", "COMMON_PHOSPHATE_L22"))
    .add_select(SelectLiteral("", "COMMON_PHOSPHATE_L33"))
)

In [13]:
# Map silicate
query_builder = (
    query_builder
    .add_select_column("Water body silicate")
    .add_select_column("Water body silicate_qc")
    .add_select_column("Water body silicate.units")
    
    .add_select(SelectColumn("Water body silicate", "COMMON_ORIGIN_SILICATE"))
    .add_select(SelectColumn("Water body silicate_qc", "COMMON_ORIGIN_SILICATE_QC"))
    .add_select(SelectLiteral("Water body silicate", "COMMON_ORIGIN_SILICATE_STANDARD_NAME"))
    .add_select(SelectColumn("Water body silicate.units", "COMMON_ORIGIN_SILICATE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::SLCAZZXX", "COMMON_ORIGIN_SILICATE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_SILICATE_P06"))
    # Per volume
    .add_select(SelectColumn("Water body silicate", "COMMON_SILICATE_PER_VOLUME"))
    .add_select(SelectColumn("Water body silicate_qc", "COMMON_SILICATE_PER_VOLUME_QC"))
    .add_select(SelectLiteral("mole_concentration_of_silicate_in_sea_water", "COMMON_SILICATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_SILICATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::SLCAZZXX", "COMMON_SILICATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_SILICATE_PER_VOLUME_P06"))
    # Per mass
    .add_select(
        SelectFunction("map_units", [
                SelectLiteral("SDN:P06::UPOX"), 
                SelectLiteral("SDN:P06::KGUM"), 
                SelectColumn("Water body silicate")
            ],
            "COMMON_SILICATE_PER_MASS")
    )
    .add_select(SelectColumn("Water body silicate_qc", "COMMON_SILICATE_PER_MASS_QC"))
    .add_select(SelectLiteral("moles_of_silicate_per_unit_mass_in_sea_water", "COMMON_SILICATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_SILICATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP012", "COMMON_SILICATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_SILICATE_PER_MASS_P06"))
    # Instruments
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_SILICATE_L05"))
    .add_select(SelectLiteral("", "COMMON_SILICATE_L22"))
    .add_select(SelectLiteral("", "COMMON_SILICATE_L33"))
)

In [14]:
# Map temperature

query_builder = (
    query_builder
    .add_select_column("ITS-90 water temperature")
    .add_select_column("ITS-90 water temperature_qc")
    .add_select_column("ITS-90 water temperature.units")
    .add_select(SelectColumn("ITS-90 water temperature", "COMMON_ORIGIN_TEMPERATURE"))
    .add_select(SelectColumn("ITS-90 water temperature_qc", "COMMON_ORIGIN_TEMPERATURE_QC"))
    .add_select(SelectLiteral("ITS-90 water temperature", "COMMON_ORIGIN_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectColumn("ITS-90 water temperature.units", "COMMON_ORIGIN_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_ORIGIN_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_ORIGIN_TEMPERATURE_P06"))
    # COMMON TEMPERATURE
    .add_select(SelectColumn("ITS-90 water temperature", "COMMON_TEMPERATURE"))
    .add_select(SelectColumn("ITS-90 water temperature_qc", "COMMON_TEMPERATURE_QC"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    # COMMON TEMPERATURE INSTRUMENTS
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_TEMPERATURE_L05"))
    .add_select(SelectLiteral("", "COMMON_TEMPERATURE_L22"))
    .add_select(SelectLiteral("", "COMMON_TEMPERATURE_L33"))
)

In [15]:
# Map salinity
query_builder = (
    query_builder
    .add_select_column("Water body salinity")
    .add_select_column("Water body salinity_qc")
    .add_select_column("Water body salinity.units")
    .add_select(SelectColumn("Water body salinity", "COMMON_ORIGIN_SALINITY"))
    .add_select(SelectColumn("Water body salinity_qc", "COMMON_ORIGIN_SALINITY_QC"))
    .add_select(SelectLiteral("Water body salinity", "COMMON_ORIGIN_SALINITY_STANDARD_NAME"))
    .add_select(SelectColumn("Water body salinity.units", "COMMON_ORIGIN_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_ORIGIN_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_ORIGIN_SALINITY_P06"))
    # COMMON SALINITY
    .add_select(SelectColumn("Water body salinity", "COMMON_SALINITY"))
    .add_select(SelectColumn("Water body salinity_qc", "COMMON_SALINITY_QC"))
    .add_select(SelectLiteral("sea_water_practical_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("dimensionless", "COMMON_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    # COMMON SALINITY INSTRUMENTS
    .add_select(SelectFunction("map_emodnet_chemistry_instrument_l05", [SelectColumn("Instrument / gear type")], "COMMON_SALINITY_L05"))
    .add_select(SelectLiteral("", "COMMON_SALINITY_L22"))
    .add_select(SelectLiteral("", "COMMON_SALINITY_L33"))
)

In [16]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1950, 2025):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("COMMON_TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    output_file = os.path.join(output_dir, f"emodnet_chemistry_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")

Processing year 1950
Running query: {"from": "default", "select": [{"function": "coalesce", "args": [{"column": "time_ISO8601", "alias": null}, {"column": "yyyy-mm-ddThh:mm:ss.sss", "alias": null}], "alias": "COMMON_TIME"}, {"column": "time_ISO8601_qc", "alias": "COMMON_TIME_QC"}, {"column": "yyyy-mm-ddThh:mm:ss.sss", "alias": null}, {"column": "Longitude", "alias": null}, {"column": "Longitude.units", "alias": null}, {"column": "Longitude", "alias": "COMMON_LONGITUDE"}, {"value": "degree_east", "alias": "COMMON_LONGITUDE_UNITS"}, {"value": "longitude", "alias": "COMMON_LONGITUDE_STANDARD_NAME"}, {"value": "SDN:P01::ALONZZ01", "alias": "COMMON_LONGITUDE_P01"}, {"value": "SDN:P06::DEGE", "alias": "COMMON_LONGITUDE_P06"}, {"column": "Latitude", "alias": null}, {"column": "Latitude.units", "alias": null}, {"column": "Latitude", "alias": "COMMON_LATITUDE"}, {"value": "degree_north", "alias": "COMMON_LATITUDE_UNITS"}, {"value": "latitude", "alias": "COMMON_LATITUDE_STANDARD_NAME"}, {"value"